## Machine Learning Tutorials with Watson Machine Learning
### Part 3 - Decision Trees 

This tutoral was adapted from the Spark documentation at https://spark.apache.org/docs/2.1.1/ml-classification-regression.html#decision-tree-classifier

### 3.1 Add Data

Before beginning, you will find it is necessary to load the text file into dsx. 
Download it from the GitHub repository and upload into dsx in the upper right hand corner "Find and Add Data"

In [ ]:
# once upload, select "sample_libsvm_data.txt"
# under Insert to Code, select Insert SparkSession Setup, place that code here



In [ ]:
#print the path created for you and copy it in the next cell


In [ ]:
newdata = spark.read.format("libsvm").load('***Copy file path here***')


### 3.2 Build Decision Tree Model pipline 
What is differs between building a Decision Tree Model and a Linear Regression Model?

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print "Test Error = %g " % (1.0 - accuracy)

treeModel = model.stages[2]
# summary only
print treeModel

### 3.3 Exercise - Watson Machine Learning
#### save this model and publish it to Watson Machine Learning 